In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import json
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pickle

## Can do vectorization if needed in following cells

In [2]:
from sklearn.decomposition import TruncatedSVD

In [3]:
vectorizer = TfidfVectorizer(stop_words='english')

X_train = vectorizer.fit_transform(string_field)
print(X_train[0])
x_train_array = X_train.toarray()

svd = TruncatedSVD(n_components=1)
#print(type(svd))
text_X_train = svd.fit_transform(X_train)
#print(text_X_train)
#print(text_X_train.shape)

for index, value in enumerate(text_X_train):
    print(string_field[index], value)

print(text_X_train)
#print(x_train_array)
print(x_train_array.shape)
print(x_train_array[0][44])
f_xtrain_nosvp = np.concatenate((song_features_array, x_train_array), axis=1)
print('xtrain_nosvp', f_xtrain_nosvp.shape)
f_xtrain = np.concatenate((song_features_array, text_X_train), axis=1)
print(f_xtrain)
print(f_xtrain.shape)

#print(X_train)
#print(f"n_samples: {X_train.shape[0]}, n_features: {X_train.shape[1]}")

NameError: name 'TfidfVectorizer' is not defined

In [ ]:
print(f"{X_train.nnz / np.prod(X_train.shape):.3f}")
print(f"About {X_train.nnz / np.prod(X_train.shape)*100:.3f}\% of the entries in the matrix are non zero")

## Reading in Spotify Song-Feature Data

In [8]:
path = 'song_data/'
def get_song_dataframe(path):
    # returns a dataframe of all of the songs in the million playlist data set
    # path - a string containing a path to the folder where the csv files are held.
    #        for example, on my machine, the folder where I ran this script also contained a folder
    #        called "song_data" which had the csv files in it. So I called get_song_dataframe('song_data/')
    #        NOTE - Make sure to include the slash!
    file_name_list = ['song.slice.' + str(i) + '-' + str(i + 49999) + '.csv' for i in range(0,2212292, 50000)] 
    file_name_list = file_name_list + ['song.slice.2250000-2262292.csv']
                        
    df_list = []
    for file_name in file_name_list:
        df_list.append(pd.read_csv(path + file_name))
    
    data = pd.concat(df_list)
    return data

In [9]:
greg_data = get_song_dataframe(path)

data = greg_data.copy()

dropping_columns = ['album_uri', 'duration_ms', 'album_name', 'num_playlist_appearances', 'time_signature']
data_columns_dropped = data.drop(dropping_columns,axis=1)
data_NaN_dropped = data_columns_dropped[~data_columns_dropped.isna().any(axis=1)]

FileNotFoundError: [Errno 2] No such file or directory: 'playlist_data/song.slice.0-49999.csv'

In [6]:
# DAataframe for the model

df_song_features = data_NaN_dropped

NameError: name 'data_NaN_dropped' is not defined

# 
# Fuzzy Clustering

## Fuzzy clustering algorithm (with no text features)

In [6]:
#check_df
#df_song_features

In [7]:
df = df_song_features
df = df.reset_index(drop=True)
#df

NameError: name 'df_song_features' is not defined

In [51]:
# Specify INPUT songs list

input_songs_indices = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]

input_songs_data = df.loc[input_songs_indices]


#print('PLAYLIST OF INPUT SONGS ARE:')
#print()
#print(check_df['playlist_name'][input_songs_indices])
#print(check_df['track_track_name'][input_songs_indices])

In [52]:
# Fuzzy Clustering algorithm (no text features)

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
import skfuzzy as fuzz

# Step 1: Load the song data
song_data = df

# Step 2: Define the input songs
#input_songs = ['White Christmas', 'Let It Snow! Let It Snow! Let It Snow!', 'Jingle Bell Rock', 'Winter Wonderland']

# Step 4: Preprocess numerical features
numerical_features = ['danceability', 'energy', 'key', 'loudness',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo'] #['artist_fraction', 'playlist_description_encoded']  # Replace with the actual numerical feature column names
all_songs_numerical = song_data[numerical_features]
input_songs_numerical = input_songs_data[numerical_features]

# Step 5: Preprocess categorical features
#categorical_features = ['track_track_name', 'playlist_name']  # Replace with the actual categorical feature column names
#all_songs_categorical = song_data[categorical_features]
#input_songs_categorical = input_songs_data[categorical_features]

# Step 6: Perform one-hot encoding on categorical features
encoder = OneHotEncoder(sparse_output=False)
#all_songs_categorical_encoded = encoder.fit_transform(all_songs_categorical)
#input_songs_categorical_encoded = encoder.transform(input_songs_categorical)

# Step 7: Combine numerical and categorical features
all_songs_combined = all_songs_numerical #np.concatenate((all_songs_numerical, all_songs_categorical_encoded), axis=1)
input_songs_combined = input_songs_numerical #np.concatenate((input_songs_numerical, input_songs_categorical_encoded), axis=1)

# Step 8: Scale the feature values
scaler = MinMaxScaler()
X_combined = scaler.fit_transform(all_songs_combined)
input_songs_combined_scaled = scaler.transform(input_songs_combined)

# Step 9: Train the fuzzy clustering model
num_clusters = 20
cntr, u, _, _, _, _, _ = fuzz.cluster.cmeans(X_combined.T, num_clusters, m=1.00001, error=0.000005, maxiter=50000)

# Step 10: Predict the degree of membership for all songs
all_songs_membership = fuzz.cluster.cmeans_predict(X_combined.T, cntr, 2, error=0.000005, maxiter=50000)[0]

# Step 11: Predict the degree of membership for the input songs
input_songs_membership = fuzz.cluster.cmeans_predict(input_songs_combined_scaled.T, cntr, 2, error=0.000005, maxiter=50000)[0]


In [53]:
print('membership matrix of input songs into different clusters:')
#input_songs_membership 

membership matrix of input songs into different clusters:


In [54]:
print('membership matrix of all songs into different clusters:')
all_songs_membership

membership matrix of all songs into different clusters:


array([[0.04592001, 0.03919565, 0.03962726, ..., 0.11421138, 0.04377225,
        0.06960027],
       [0.01794434, 0.0165275 , 0.01357147, ..., 0.0216464 , 0.02614141,
        0.03811212],
       [0.01596682, 0.01521115, 0.01369936, ..., 0.01676405, 0.0356389 ,
        0.188265  ],
       ...,
       [0.02754536, 0.02383162, 0.02587571, ..., 0.02747233, 0.02586543,
        0.0333093 ],
       [0.02614811, 0.02518615, 0.01889078, ..., 0.01807295, 0.02957102,
        0.0260883 ],
       [0.04834239, 0.03538452, 0.06668204, ..., 0.07134028, 0.03378018,
        0.05901689]])

In [55]:
# membership cluster ranking for all songs(clusters get distant from left to right)

all_songs_membership_array = []
for song_membership in all_songs_membership.T:
    all_sorted_indices = np.argsort(song_membership)[::-1]
    all_songs_membership_array.append(all_sorted_indices)

print('membership cluster ranking for all songs(clusters get distant from left to right):')
all_songs_membership_array

membership cluster ranking for all songs(clusters get distant from left to right):


[array([10,  9, 13, 15, 19,  0, 12,  6, 14,  8,  7, 17, 18,  1,  5, 11,  3,
         2, 16,  4]),
 array([10,  9, 13, 15, 12,  8,  7, 14,  0,  6, 19, 18, 17, 11,  1,  2,  5,
         3,  4, 16]),
 array([ 9, 10, 15, 19,  0, 13, 12,  8,  6, 17, 14,  7, 18,  5, 11,  2,  1,
         3, 16,  4]),
 array([10,  9, 13,  0, 14, 19, 15,  6, 12,  8,  7, 17, 18,  5, 11,  1,  3,
         2, 16,  4]),
 array([ 9, 19, 10, 15,  8,  0,  6, 17, 13, 12,  7, 14,  5, 11, 18,  1,  2,
         3, 16,  4]),
 array([ 9, 10, 15, 19,  0, 13,  6, 12,  8, 17, 14,  7, 18,  5, 11,  1,  3,
         2, 16,  4]),
 array([10,  6,  9,  0, 14, 19, 13, 15, 12,  5,  3,  8,  7, 11, 18, 17,  1,
         2,  4, 16]),
 array([15, 19,  9,  8, 10, 12,  6,  0,  7, 13, 17, 14, 11,  5, 18,  2,  3,
         1, 16,  4]),
 array([10,  9, 13,  0, 14,  6, 19, 15, 12,  8,  7, 11, 18,  5,  3, 17,  1,
         2,  4, 16]),
 array([10,  9, 13, 15,  0, 12,  8, 14, 19,  7,  6, 17, 18, 11,  1,  5,  2,
         3, 16,  4]),
 array([13, 10, 12, 

In [56]:
# cluster ranking for each input song
print('cluster ranking for each input tarck:')
print('')
i = 0
cluster_num_string_array = []
sorted_indices_array = []

for input_membership_array in input_songs_membership.T:
    sorted_indices = np.argsort(input_membership_array)[::-1]
    sorted_indices_array.append(sorted_indices)
    
    cluster_num_string = "Cluster ranks for Input Track {}:    ".format(i)
    cluster_num_string_array.append(cluster_num_string)
    
    i=i+1
    
test_encoding = tuple(zip(cluster_num_string_array , sorted_indices_array))

for cluster_rankings in test_encoding:
    cluster_label = cluster_rankings[0]
    cluster_array = cluster_rankings[1].tolist()
    print(cluster_label, cluster_array)

cluster ranking for each input tarck:

Cluster ranks for Input Track 0:     [10, 9, 13, 15, 12, 8, 7, 14, 0, 6, 19, 18, 17, 11, 1, 2, 5, 3, 4, 16]
Cluster ranks for Input Track 1:     [9, 10, 15, 19, 0, 13, 12, 8, 6, 17, 14, 7, 18, 5, 11, 2, 1, 3, 16, 4]
Cluster ranks for Input Track 2:     [10, 9, 13, 0, 14, 19, 15, 6, 12, 8, 7, 17, 18, 5, 11, 1, 3, 2, 16, 4]
Cluster ranks for Input Track 3:     [9, 19, 10, 15, 8, 0, 6, 17, 13, 12, 7, 14, 5, 11, 18, 1, 2, 3, 16, 4]
Cluster ranks for Input Track 4:     [9, 10, 15, 19, 0, 13, 6, 12, 8, 17, 14, 7, 18, 5, 11, 1, 3, 2, 16, 4]
Cluster ranks for Input Track 5:     [10, 6, 9, 0, 14, 19, 13, 15, 12, 5, 3, 8, 7, 11, 18, 17, 1, 2, 4, 16]
Cluster ranks for Input Track 6:     [15, 19, 9, 8, 10, 12, 6, 0, 7, 13, 17, 14, 11, 5, 18, 2, 3, 1, 16, 4]
Cluster ranks for Input Track 7:     [10, 9, 13, 0, 14, 6, 19, 15, 12, 8, 7, 11, 18, 5, 3, 17, 1, 2, 4, 16]
Cluster ranks for Input Track 8:     [10, 9, 13, 15, 0, 12, 8, 14, 19, 7, 6, 17, 18, 11, 1, 5, 2,

In [57]:
# final cluster ranking by considering distance of all songs to all clusters

input_songs_membership_sum = np.sum(input_songs_membership.T, axis=0)
input_songs_membership_sum

input_songs_membership_sorted_indices = np.argsort(input_songs_membership_sum)[::-1]
print('Final cluster ranking by considering distance from each input track to each cluster center:')
print('')
print(input_songs_membership_sorted_indices)

Final cluster ranking by considering distance from each input track to each cluster center:

[10  9 13 15 14 19  6  0 12  8  7 17 18 11  5  3  1  2  4 16]


In [62]:
# finding best matching track

refinement = 5

target_array = input_songs_membership_sorted_indices[0:refinement]

matching_index = None
matching_index_array = []

# Iterating over the array list
for index, arr in enumerate(all_songs_membership_array):
    # Checking if the first two entries match

    if np.array_equal(arr[:refinement], target_array):
        matching_index = index
        matching_index_array.append(matching_index)
        #break  # Exit the loop after finding the first match

# Printing the matching index
    if matching_index is not None:
        print("Matching index:", matching_index)
        print("Matching Track:", song_data["track_name"][matching_index])
        #print("Playlist of matching Track:", check_df['playlist_name'][matching_index])
        
        print('')
    #else:
        #print("No match found.")
    matching_index = None
if matching_index is None:
    print("No match found.")

Matching index: 434
Matching Track: Dynamite (feat. Allday)

Matching index: 3202
Matching Track: Everything Is Not As It Seems - Series Theme Song

Matching index: 7001
Matching Track: So Fresh, So Clean

Matching index: 8358
Matching Track: Two Way Love Affair

Matching index: 11695
Matching Track: Touchdown (Island Love Version)

Matching index: 11772
Matching Track: What Was I Thinkin'

Matching index: 13667
Matching Track: (If You're Not In It For Love) I'm Outta Here!

Matching index: 15029
Matching Track: Mayores

Matching index: 17468
Matching Track: Let's Ride

Matching index: 26920
Matching Track: Put It Down

Matching index: 31995
Matching Track: Drinkin' Bone

Matching index: 32813
Matching Track: All Of The People

Matching index: 37981
Matching Track: Margarita

Matching index: 45151
Matching Track: Back & Forth

Matching index: 46863
Matching Track: Something Happened On The Way To Heaven - 2016 Remastered

Matching index: 56673
Matching Track: Side Chick

Matching index

In [63]:
matching_track_names = song_data["track_name"][matching_index_array]
#matching_playlist_names = song_data['playlist_name'][matching_index_array]

In [64]:
# Best song out all all predictions

all_song_cumulative_distance = []

for each_index in matching_index_array:
    each_track_distances = all_songs_membership[:,each_index]
    cumulative_distance = np.sum(each_track_distances[0:refinement])
    all_song_cumulative_distance.append(cumulative_distance)
    

In [65]:
requred_recommendations = 20

distance_array_accending_indexes = np.argsort(all_song_cumulative_distance)  # np.argpartition(all_song_cumulative_distance) #np.argsort(all_song_cumulative_distance)[:requred_recommendations]

recommendation_indexes = [matching_index_array[i] for i in distance_array_accending_indexes][:requred_recommendations]

for each_song_index in recommendation_indexes:
    #print('Recommended song:', song_data["playlist_name"][each_song_index])
    print('Recommended song:', song_data["track_name"][each_song_index])
    print(each_song_index)

Recommended song: Corrido De Los Sanchez
1394318
Recommended song: Smart, Smart, Smart
996687
Recommended song: 데자-부 (Déjà-Boo)
692701
Recommended song: Sunny
1872031
Recommended song: Siempre Que Me Emborracho
1265069
Recommended song: Las Maracas
1473973
Recommended song: Why Do I Do
317164
Recommended song: Mining Craft
2065905
Recommended song: Just a Closer Walk / How Great thou Art
1152797
Recommended song: La Probadita
2008128
Recommended song: Folly
794755
Recommended song: Gotta Serve Somebody
390593
Recommended song: The way you make me melt
1619377
Recommended song: 每天 Forever with You - sung by SUPER JUNIOR-M
785643
Recommended song: Isle Of Paradise - Radio Edit
1340530
Recommended song: Music Is Alive
1058498
Recommended song: Rumba rendo
899216
Recommended song: In My Head - Radio Edit
436616
Recommended song: Glow
2031003
Recommended song: Hay Un Amor Afuera
1664521


In [25]:
# Best song out all all predictions

all_song_cumulative_distance = []

for each_index in matching_index_array:
    each_track_distances = all_songs_membership[:,each_index]
    cumulative_distance = np.sum(each_track_distances[0:refinement])
    all_song_cumulative_distance.append(cumulative_distance)
    

In [26]:
requred_recommendations = 3

recommendation_indexes = np.argsort(all_song_cumulative_distance)[:requred_recommendations]

for each_song_index in recommendation_indexes:
    print('Recommended song:')

Recommended song:
Recommended song:
Recommended song:


In [27]:
recommendation_indexes

array([17015,  6981, 25111])

In [28]:
len(all_song_cumulative_distance)

91253

## Fuzzy clustering algorithm (with text features)

In [ ]:
# DF Preparation (with text_data)

numerical_features = ['danceability', 'energy', 'key', 'loudness',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'song_popularity', 'artist_popularity', 'album_popularity']

all_features = numerical_features + ['text_data']
df_with_text = pd.DataFrame(np.concatenate((df_song_features, text_X_train), axis=1), columns=all_features)

df = df_with_text

In [ ]:
# Specify INPUT songs list

input_songs_indices = [1, 212, 213, 214, 215, 216, 217, 218, 200, 300, 400, 500, 514]

input_songs_data = df.loc[input_songs_indices]
print('PLAYLIST OF INPUT SONGS ARE:')
print()
print(check_df['playlist_name'][input_songs_indices])
#print(check_df['track_track_name'][input_songs_indices])

In [ ]:
# Fuzzy Clustering algorithm (with text features)

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
import skfuzzy as fuzz

# Step 1: Load the song data
song_data = df

# Step 2: Define the input songs
#input_songs = ['White Christmas', 'Let It Snow! Let It Snow! Let It Snow!', 'Jingle Bell Rock', 'Winter Wonderland']

# Step 4: Preprocess numerical features
numerical_features = ['danceability', 'energy', 'key', 'loudness',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'song_popularity', 'artist_popularity', 'album_popularity', 'text_data'] #['artist_fraction', 'playlist_description_encoded']  # Replace with the actual numerical feature column names

all_songs_numerical = song_data[numerical_features]
input_songs_numerical = input_songs_data[numerical_features]

# Step 5: Preprocess categorical features
#categorical_features = ['text_data']  # Replace with the actual categorical feature column names
#all_songs_categorical = song_data[categorical_features]
#input_songs_categorical = input_songs_data[categorical_features]

# Step 6: Perform one-hot encoding on categorical features
encoder = OneHotEncoder(sparse_output=False)
#all_songs_categorical_encoded = encoder.fit_transform(all_songs_categorical)
#input_songs_categorical_encoded = encoder.transform(input_songs_categorical)

# Step 7: Combine numerical and categorical features
all_songs_combined = all_songs_numerical #np.concatenate((all_songs_numerical, all_songs_categorical_encoded), axis=1)
input_songs_combined = input_songs_numerical #np.concatenate((input_songs_numerical, input_songs_categorical_encoded), axis=1)

# Step 8: Scale the feature values
scaler = MinMaxScaler()
X_combined = scaler.fit_transform(all_songs_combined)
input_songs_combined_scaled = scaler.transform(input_songs_combined)

# Step 9: Train the fuzzy clustering model
num_clusters = 7
cntr, u, _, _, _, _, _ = fuzz.cluster.cmeans(X_combined.T, num_clusters, m=1.00001, error=0.000005, maxiter=50000)

# Step 10: Predict the degree of membership for all songs
all_songs_membership = fuzz.cluster.cmeans_predict(X_combined.T, cntr, 2, error=0.000005, maxiter=50000)[0]

# Step 11: Predict the degree of membership for the input songs
input_songs_membership = fuzz.cluster.cmeans_predict(input_songs_combined_scaled.T, cntr, 2, error=0.000005, maxiter=50000)[0]


In [ ]:
print('membership matrix of input songs into different clusters:')
#input_songs_membership 

In [ ]:
print('membership matrix of all songs into different clusters:')
#all_songs_membership

In [ ]:
# membership cluster ranking for all songs(clusters get distant from left to right)

all_songs_membership_array = []
for song_membership in all_songs_membership.T:
    all_sorted_indices = np.argsort(song_membership)[::-1]
    all_songs_membership_array.append(all_sorted_indices)

print('membership cluster ranking for all songs(clusters get distant from left to right):')
#all_songs_membership_array

In [ ]:
# cluster ranking for each input song
print('cluster ranking for each input tarck:')
print('')
i = 0
cluster_num_string_array = []
sorted_indices_array = []

for input_membership_array in input_songs_membership.T:
    sorted_indices = np.argsort(input_membership_array)[::-1]
    sorted_indices_array.append(sorted_indices)
    
    cluster_num_string = "Cluster ranks for Input Track {}:    ".format(i)
    cluster_num_string_array.append(cluster_num_string)
    
    i=i+1
    
test_encoding = tuple(zip(cluster_num_string_array , sorted_indices_array))

for cluster_rankings in test_encoding:
    cluster_label = cluster_rankings[0]
    cluster_array = cluster_rankings[1].tolist()
    print(cluster_label, cluster_array)

In [ ]:
# final cluster ranking by considering distance of all songs to all clusters

input_songs_membership_sum = np.sum(input_songs_membership.T, axis=0)
input_songs_membership_sum

input_songs_membership_sorted_indices = np.argsort(input_songs_membership_sum)[::-1]
print('Final cluster ranking by considering distance from each input track to each cluster center:')
print('')
print(input_songs_membership_sorted_indices)

In [ ]:
# finding the best matching track

refinement = 2

target_array = input_songs_membership_sorted_indices[0:refinement]

matching_index = None

# Iterating over the array list
for index, arr in enumerate(all_songs_membership_array):
    # Checking if the first two entries match

    if np.array_equal(arr[:refinement], target_array):
        matching_index = index
        #break  # Exit the loop after finding the first match

# Printing the matching index
    if matching_index is not None:
        #print("Matching index:", matching_index)
        #print("Matching Track:", check_df["track_track_name"][matching_index])
        print("Playlist of matching Track:", check_df['playlist_name'][matching_index])
        
        print('')
    #else:
        #print("No match found.")
    matching_index = None
if matching_index is None:
    print("No match found.")